#2

In [39]:
class Record:
    def __init__(self, **kwargs) -> None:
        self.__dict__.update(kwargs)
        self.lst = list(kwargs)

    def __getitem__(self, item):
        if type(item) is int and 0 <= item < len(self.lst):
            return self.__dict__[self.lst[item]]
        raise IndexError('неверный индекс поля')

    def __setitem__(self, key, value):
        if type(key) is int and 0 <= key < len(self.lst):
            self.__dict__[self.lst[key]] = value
        else:
            raise IndexError('неверный индекс поля')

In [40]:
r = Record(pk=1, title='Python ООП', author='Балакирев')
r[0] = 2 # доступ к полю pk
r[1] = 'Супер курс по ООП' # доступ к полю title
r[2] = 'Балакирев С.М.' # доступ к полю author
print(r[1]) # Супер курс по ООП
r[3] # генерируется исключение IndexError

Супер курс по ООП


IndexError: неверный индекс поля

#3

In [51]:
class Track:
    def __init__(self, start_x, start_y) -> None:
        self.start_x = start_x
        self.start_y = start_y
        self.points = []

    def add_point(self, x, y, speed):
        self.points.append([x, y, speed])

    def __getitem__(self, indx):
        if type(indx) is int and 0 <= indx < len(self.points):
            return (self.points[indx][0], self.points[indx][1]), self.points[indx][2]
        raise IndexError('некорректный индекс')

    def __setitem__(self, indx, speed):
        if type(indx) is int and 0 <= indx < len(self.points):
            self.points[indx][2] = speed
        else:
            raise IndexError('некорректный индекс')

In [52]:
tr = Track(10, -5.4)
tr.add_point(20, 0, 100) # первый линейный сегмент: indx = 0
tr.add_point(50, -20, 80) # второй линейный сегмент: indx = 1
tr.add_point(63.45, 1.24, 60.34) # третий линейный сегмент: indx = 2

tr[2] = 60
c, s = tr[2]
print(c, s)

res = tr[3] # IndexError


(63.45, 1.24) 60


IndexError: некорректный индекс

#4

In [106]:
class Integer:
    def __init__(self, start_value) -> None:
        self.value = start_value

    @property
    def value(self):
        return self.__value

    @value.setter
    def value(self, value):
        if type(value) is not int:
            raise ValueError('должно быть целое число')
        self.__value = value

class Array:
    def __init__(self, max_length, cell) -> None:
        self.array = [cell(0) for x in range(max_length)]

    def __getitem__(self, indx):
        if type(indx) is int and 0 <= indx < len(self.array):
            return self.array[indx].value
        raise IndexError('неверный индекс для доступа к элементам массива')

    def __setitem__(self, indx, value):
        if type(indx) is int and 0 <= indx < len(self.array):
            self.array[indx].value = value
        else:
            raise IndexError('неверный индекс для доступа к элементам массива')

    def __repr__(self) -> str:
        return ' '.join([str(x.value) for x in self.array])

In [108]:
ar_int = Array(10, cell=Integer)
print(ar_int[3])
print(ar_int) # должны отображаться все значения массива в одну строчку через пробел
ar_int[1] = 10
ar_int[1] = 10.5 # должно генерироваться исключение ValueError
ar_int[10] = 1 # должно генерироваться исключение IndexError


0
0 0 0 0 0 0 0 0 0 0


ValueError: должно быть целое число

#5

In [127]:
class IntegerValue:
    def __set_name__(self, owner, name):
        self.name = '_' + name

    def __get__(self, instance, owner):
        return getattr(instance, self.name)

    def __set__(self, instance, value):
        if type(value) is not int:
            raise ValueError('возможны только целочисленные значения')
        setattr(instance, self.name, value)

class CellInteger:
    value = IntegerValue()

    def __init__(self, start_value=0) -> None:
        self.value = start_value

class TableValues:
    def __init__(self, rows, cols, cell=CellInteger) -> None:
        if cell is None:
            raise ValueError('параметр cell не указан')
        self.type = cell
        self.cells = [[cell() for x in range(cols)] for y in range(rows)]

    def __getitem__(self, indx):
        return self.cells[indx[0]][indx[1]].value

    def __setitem__(self, indx, value):
        self.cells[indx[0]][indx[1]] = self.type(value)

        


In [128]:
table = TableValues(2, 3, cell=CellInteger)
print(table[0, 1])
table[1, 1] = 10
table[0, 0] = 1.45


0


ValueError: возможны только целочисленные значения

In [129]:
# вывод таблицы в консоль
for row in table.cells:
    for x in row:
        print(x.value, end=' ')
    print()


0 0 0 
0 10 0 


#7

In [173]:
class StackObj:
    def __init__(self, data) -> None:
        self.data = data
        self.next = None
        self.prev = None

class Stack:
    def __init__(self) -> None:
        self.top = None
        self.last = None

    def push(self, obj):
        if self.top is None:
            self.top = obj
            self.last = obj
        else:
            obj.prev = self.last
            self.last.next = obj
            self.last = obj

    def pop(self):
        prev = self.last.prev
        self.last.prev = None
        prev.next = None
        to_return = self.last
        self.last = prev
        return to_return

    def __getitem__(self, indx):
        curr_indx = 0
        curr_obj = self.top

        while curr_indx != indx:
            curr_obj = curr_obj.next
            curr_indx += 1

        if curr_obj is None:
            raise IndexError('неверный индекс')

        return curr_obj

    def __setitem__(self, indx, value):
        curr_obj = self.__getitem__(indx)
        curr_obj.prev.next = value
        curr_obj.next.prev = value
        value.next = curr_obj.next
        value.prev = curr_obj.prev
        curr_obj.prev = None
        curr_obj.next = None
    


In [172]:
st = Stack()
st.push(StackObj("obj1"))
st.push(StackObj("obj2"))
st.push(StackObj("obj3"))
st[1] = StackObj("new obj2")
print(st[2].data) # obj3
print(st[1].data) # new obj2
res = st[3] # исключение IndexError


obj3
new obj2


IndexError: неверный индекс

#7

In [184]:
class RadiusVector:
    def __init__(self, *args) -> None:
        self.coords = list(args)

    def __getitem__(self, req_slice):
        return tuple(self.coords[req_slice]) if type(req_slice) is slice else self.coords[req_slice]
        
    def __setitem__(self, req_slice, values):
        self.coords[req_slice] = values

In [185]:
v = RadiusVector(1, 1, 1, 1)
print(v[1]) # 1
v[:] = 1, 2, 3, 4
print(v[2]) # 3
print(v[1:]) # (2, 3, 4)
v[0] = 10.5


1
3
(2, 3, 4)


#8

In [195]:
class Cell:
    def __init__(self) -> None:
        self.is_free = True
        self.value = 0

    def __bool__(self):
        return self.is_free

class TicTacToe:
    def __init__(self) -> None:
        self.pole = tuple(tuple(Cell() for x in range(3)) for y in range(3))

    def clear(self):
        self.pole = tuple(tuple(Cell() for x in range(3)) for y in range(3))

    def __check(self, item):
        if type(item) != tuple or len(item) != 2:
            raise IndexError('неверный индекс клетки')
        if any(not (0 <= x < 3) for x in item if type(x) != slice):
            raise IndexError('неверный индекс клетки')

    def __setitem__(self, key, value):
        self.__check(key)
        r, c = key
        if self.pole[r][c]:
            self.pole[r][c].value = value
            self.pole[r][c].is_free = False
        else:
            raise ValueError('клетка уже занята')

    def __getitem__(self, key):
        self.__check(key)
        r, c = key
        if type(r) == slice:
            return tuple(self.pole[x][c].value for x in range(3))
        if type(c) == slice:
            return tuple(self.pole[r][x].value for x in range(3))
        
        return self.pole[r][c].value



#9

In [210]:
class Thing:
    def __init__(self, name, weight) -> None:
        self.name = name
        self.weight = weight

class Bag:
    def __init__(self, max_weight) -> None:
        self.max_weight = max_weight
        self.things = []
        self.curr_weight = 0

    def add_thing(self, thing):
        if self.curr_weight + thing.weight <= self.max_weight:
            self.things.append(thing)
            self.curr_weight += thing.weight
        else:
            raise ValueError('превышен суммарный вес предметов')

    def __check(self, value):
        if not 0 <= value < len(self.things):
            raise IndexError('неверный индекс')

    def __getitem__(self, indx):
        self.__check(indx)
        return self.things[indx]

    def __delitem__(self, indx):
        self.__check(indx)
        item = self.things.pop(indx)
        self.curr_weight -= item.weight

    def __setitem__(self, indx, value):
        self.__check(indx)
        if self.curr_weight - self.things[indx].weight + value.weight <= self.max_weight:
            item = self.things.pop(indx)
            self.curr_weight -= item.weight
            self.things.insert(indx, value)
            self.curr_weight += value.weight
        else:
            raise ValueError('превышен суммарный вес предметов')

In [209]:
bag = Bag(1000)
bag.add_thing(Thing('книга', 100))
bag.add_thing(Thing('носки', 200))
bag.add_thing(Thing('рубашка', 500))
print(bag[2].name) # рубашка
bag[1] = Thing('платок', 100)
print(bag[1].name) # платок
del bag[0]
print(bag[0].name) # платок


рубашка
платок
платок


#10

In [253]:
class Cell:
    def __init__(self, value) -> None:
        self.value = value

class SparseTable:
    def __init__(self) -> None:
        self.rows = 0
        self.cols = 0
        self.cells = {}

    def update_size(self):
        self.rows = max(key[0] for key in self.cells) + 1
        self.cols = max(key[1] for key in self.cells) + 1


    def add_data(self, row, col, data):
        self.cells.setdefault((row, col), data)
        self.update_size()
    
    def remove_data(self, row, col):
        try:
            del self.cells[(row, col)]
            self.update_size()
        except KeyError:
            raise IndexError('ячейка с указанными индексами не существует')

    def __getitem__(self, indx):
        try:
            return self.cells[indx].value
        except KeyError:
            raise ValueError('данные по указанным индексам отсутствуют')

    def __setitem__(self, indx, value):
        self.cells[indx] = Cell(value)
        self.update_size()

In [254]:
st = SparseTable()
st.add_data(2, 5, Cell("cell_25"))
st.add_data(0, 0, Cell("cell_00"))
st[11, 7] = 'cell_117'
st.remove_data(11, 7)

In [252]:
st.rows

3

In [255]:
st = SparseTable()
st.add_data(2, 5, Cell("cell_25"))
st.add_data(0, 0, Cell("cell_00"))
st[2, 5] = 25
st[11, 7] = 'cell_117'
print(st[0, 0]) 
st.remove_data(2, 5)
print(st.rows, st.cols) 

cell_00
11 7


In [229]:
st.remove_data(12, 3) 

IndexError: ячейка с указанными индексами не существует